In [20]:
import pandas as pd
import json
from sqlalchemy import create_engine
from selenium import webdriver
from mysql_con import con

# creat date range
date_list = pd.date_range(end=pd.datetime.today(), periods=3650).tolist()

# required function
def vegCrawler(date_target):
    try:
        if date_target.year > 2010:
            date_RE_target = "%s%s" % (str(date_target.year-1911), date_target.strftime("/%m/%d")) 
        else:
            date_RE_target = "0%s%s" % (str(date_target.year-1911), date_target.strftime("/%m/%d"))
        # date select
        driver.find_element_by_name("ctl00$contentPlaceHolder$txtSTransDate").click()
        driver.execute_script("setValue('%s');" % date_RE_target)
        # submit
        driver.find_element_by_name("ctl00$contentPlaceHolder$btnQuery").click()
        time.sleep(3)
        # find table
        res = driver.find_element_by_xpath("//div[@id='ctl00_contentPlaceHolder_panel']")
        content = res.get_attribute('innerHTML')
        print("Crawler state " + date_target.strftime("%Y/%m/%d") + " complete!")
        return content
    except Exception as e:
        driver.switch_to.alert.accept()
        print(e)

def vegTransform(content, date_target):
    try:
        tmp = pd.read_html(content, header=0)[2]
        df = pd.DataFrame(tmp).drop(0)
        df.dropna(inplace=True)

        # clean column_1
        str1 = [i.split(" ")[0] for i in df.iloc[:,0]]
        df.iloc[:,0] = [int(i) for i in str1]
        # clean column_2
        df.iloc[:,1] = [i.split(" ")[0] for i in df.iloc[:,1]]

        df.drop(df.columns[[6,8]], axis=1, inplace=True)
        df.columns = ["market", "produce", "price_top", "price_mid", "price_low", "price_ave", "volumn"]
        df["date_"] = date_target.strftime("%Y/%m/%d")

        print("Transform state " + date_target.strftime("%Y/%m/%d") + " complete!")
        return df
    except Exception as e:
        print(e)

def vegMySQL(df, con):
    try:
        df.to_sql('price',con=con(),if_exists='append',index=False)
        print("DB store complete!")
    except Exception as e:
        print(e)

In [17]:
# initialize
driver = webdriver.Chrome("/Users/timchen/Documents/chromedriver")
driver.get("http://amis.afa.gov.tw/main/Main.aspx")
driver.find_element_by_xpath("//a[@class='menu-01']").click() # menu
driver.find_element_by_xpath("//li[1]/a[@class='map_menu_layer01_nopic']").click() # topic
driver.find_element_by_xpath("//textarea[@id='ctl00_contentPlaceHolder_txtMarket']").click() # produce
driver.switch_to.frame(driver.find_element_by_tag_name("iframe")) # switch frame
driver.find_element_by_id("radlMarketRange_0").click() # all produce
driver.find_element_by_name("ctl00$contentPlaceHolder$txtProduct").click() # market
driver.switch_to.frame(driver.find_element_by_tag_name("iframe")) # switch frame
driver.find_element_by_id("radlProductType_0").click() # all market

In [45]:
import time
for i,j in enumerate(date_list[3583:]):
    tmp_content = vegCrawler(j)
    tmp_df = vegTransform(tmp_content, j)
    vegMySQL(tmp_df, con=engine)

Crawler state 2018/06/16 complete!
Transform state 2018/06/16 complete!
DB store complete!
Crawler state 2018/06/17 complete!
Transform state 2018/06/17 complete!
DB store complete!
Crawler state 2018/06/18 complete!
Transform state 2018/06/18 complete!
DB store complete!
Crawler state 2018/06/19 complete!
Transform state 2018/06/19 complete!
DB store complete!
Crawler state 2018/06/20 complete!
Transform state 2018/06/20 complete!
DB store complete!
Crawler state 2018/06/21 complete!
Transform state 2018/06/21 complete!
DB store complete!
Crawler state 2018/06/22 complete!
Transform state 2018/06/22 complete!
DB store complete!
Crawler state 2018/06/23 complete!
Transform state 2018/06/23 complete!
DB store complete!
Crawler state 2018/06/24 complete!
Transform state 2018/06/24 complete!
DB store complete!
Crawler state 2018/06/25 complete!
Transform state 2018/06/25 complete!
DB store complete!
Crawler state 2018/06/26 complete!
Transform state 2018/06/26 complete!
DB store complete!

In [46]:
driver.close

<bound method WebDriver.close of <selenium.webdriver.chrome.webdriver.WebDriver (session="af296e94d673310bd896f89dea3b8cf3")>>